### Load tensorflow

# New Section

In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [0]:
from google.colab import drive

In [5]:
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!ls "/content/gdrive/My Drive/Iris.csv"

'/content/gdrive/My Drive/Iris.csv'


In [0]:
import io
from google.colab import drive
prices_df = pd.read_csv('./prices.csv')
#iris_df= pd.read_csv('/content/gdrive/My Drive/Iris.csv')

In [8]:
from google.colab import files
uploaded = files.upload()

Saving prices.csv to prices.csv


### Check all columns in the dataset

In [13]:
prices_df.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
prices_df = prices_df.drop(["date", "symbol"], axis=1)

In [15]:
prices_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data = prices_df.iloc[:1000]

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float64
close     1000 non-null float64
low       1000 non-null float64
high      1000 non-null float64
volume    1000 non-null float64
dtypes: float64(5)
memory usage: 39.1 KB


### Divide the data into train and test sets

In [18]:
# splitting data into training and test set for independent attributes
from sklearn.model_selection import train_test_split

X =  data.drop(["close"], axis=1)
y =  data['close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

((800, 4), (800,))
((200, 4), (200,))


## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
# Define Input place holder
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

x_n = tf.layers.batch_normalization(x, training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [24]:
X_train.dtypes

open      float64
low       float64
high      float64
volume    float64
dtype: object

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [26]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:X_train, 
                                                        y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 7195.4736)
('Training loss at step: ', 10, ' is ', 5777.4297)
('Training loss at step: ', 20, ' is ', 4829.152)
('Training loss at step: ', 30, ' is ', 2929.7683)
('Training loss at step: ', 40, ' is ', 2929.5615)
('Training loss at step: ', 50, ' is ', 2929.4717)
('Training loss at step: ', 60, ' is ', 2929.4194)
('Training loss at step: ', 70, ' is ', 2929.386)
('Training loss at step: ', 80, ' is ', 2929.3635)
('Training loss at step: ', 90, ' is ', 2929.348)


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [27]:
sess.run(W)

array([[ 0.12530866],
       [ 1.0305027 ],
       [-6.0524473 ],
       [ 0.29347968]], dtype=float32)

In [28]:
sess.run(b)

array([26.89262], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [0]:
_, test_loss = sess.run([train_op, loss],feed_dict={x:X_test, 
                                                        y_:y_test})

In [30]:
Abs_MS_Diff_loss = train_loss - test_loss
Abs_MS_Diff_loss

-2376.8503

In [31]:
X.shape

(1000, 4)

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape =(4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(1))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [33]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=30,
          batch_size=X_train.shape[0])

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 0s 300us/step - loss: 7234.6187 - val_loss: 3496.0996
Epoch 2/30
800/800 [==============================] - 0s 12us/step - loss: 6646.1763 - val_loss: 552.8278
Epoch 3/30
800/800 [==============================] - 0s 6us/step - loss: 4579.8540 - val_loss: 147991.5469
Epoch 4/30
800/800 [==============================] - 0s 8us/step - loss: 208.8797 - val_loss: 283927.0312
Epoch 5/30
800/800 [==============================] - 0s 11us/step - loss: 96.2416 - val_loss: 80831.6172
Epoch 6/30
800/800 [==============================] - 0s 6us/step - loss: 218.1567 - val_loss: 240153.9219
Epoch 7/30
800/800 [==============================] - 0s 8us/step - loss: 608.5405 - val_loss: 5948.5474
Epoch 8/30
800/800 [==============================] - 0s 10us/step - loss: 2660.8022 - val_loss: 187749.0000
Epoch 9/30
800/800 [==============================] - 0s 6us/step - loss: 1390.3517 - val_loss: 19

In [35]:
from google.colab import files
uploaded = files.upload()

Saving Iris.csv to Iris.csv


In [0]:
import io
from google.colab import drive
iris_df = pd.read_csv('./Iris.csv')

In [38]:
iris_df.head(5)
print(iris_df.columns)
iris_df = pd.get_dummies(iris_df,prefix=['Species'])

Index([u'Id', u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm',
       u'PetalWidthCm', u'Species'],
      dtype='object')


In [40]:
iris_df.columns

Index([u'Id', u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm',
       u'PetalWidthCm', u'Species_Iris-setosa', u'Species_Iris-versicolor',
       u'Species_Iris-virginica'],
      dtype='object')

In [0]:
y = iris_df[['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica']]
X = iris_df.drop(['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica','Id'],axis =1)

### Classification using Keras 

### Splitting the data into feature set and target set

In [0]:
X = iris_df.drop(['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica','Id'],axis =1)
y = iris_df[['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica']]

### Divide the dataset into Training and test (70:30)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape = (4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

In [0]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu'))
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='relu'))
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [47]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=30,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/30
105/105 [==============================] - 0s 4ms/step - loss: 1.1188 - acc: 0.3810 - val_loss: 1.1704 - val_acc: 0.3111
Epoch 2/30
105/105 [==============================] - 0s 217us/step - loss: 1.1030 - acc: 0.3429 - val_loss: 1.1944 - val_acc: 0.3111
Epoch 3/30
105/105 [==============================] - 0s 205us/step - loss: 1.0928 - acc: 0.4095 - val_loss: 1.2031 - val_acc: 0.3111
Epoch 4/30
105/105 [==============================] - 0s 268us/step - loss: 1.0840 - acc: 0.4095 - val_loss: 1.2140 - val_acc: 0.3111
Epoch 5/30
105/105 [==============================] - 0s 211us/step - loss: 1.0763 - acc: 0.4286 - val_loss: 1.2058 - val_acc: 0.3111
Epoch 6/30
105/105 [==============================] - 0s 234us/step - loss: 1.0629 - acc: 0.5333 - val_loss: 1.2074 - val_acc: 0.3111
Epoch 7/30
105/105 [==============================] - 0s 216us/step - loss: 1.0489 - acc: 0.5619 - val_loss: 1.2071 - val_acc: 0.3111
Epoch 8/30
105/105 

### Report Accuracy of the predicted values

In [48]:
scores = model.evaluate(X_test,y_test)
scores

45/45 [==============================] - 0s 180us/step


[0.699523909886678, 0.6222222261958652]